# 🚀 Autonomous Multi-Agent System on Google Colab GPU

**Version**: 2.0  
**GPU**: A100 (40GB) recommended  
**Purpose**: Deploy V1.0 Lightweight with high-performance computing  

---

## 📋 What This Notebook Does

1. ✅ Authenticates and mounts Google Drive
2. ✅ Uploads project files from Drive
3. ✅ Installs dependencies
4. ✅ Configures GPU environment
5. ✅ Starts autonomous coordination system
6. ✅ Monitors deployment progress
7. ✅ Downloads results back to Drive

---

## ⚡ Quick Start

**Step 1**: Select Runtime → Change runtime type → A100 GPU  
**Step 2**: Run all cells (Runtime → Run all)  
**Step 3**: Monitor progress in output cells  

---

## 🔧 Step 1: Environment Setup

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Authenticate and mount Google Drive
from google.colab import auth, drive

print("🔐 Authenticating Google account...")
auth.authenticate_user()
print("✅ Authentication successful")

print("\n💾 Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ Drive mounted at /content/drive")

In [ ]:
# Setup paths
import os
from pathlib import Path

# Try to auto-detect project location on Drive
print("🔍 Auto-detecting project location on Google Drive...\n")

DRIVE_BASE = Path("/content/drive/MyDrive")

# First, let's see what's actually in the most common locations
print("📂 Checking common locations:\n")

locations_to_show = [
    DRIVE_BASE / "cv_multimodal/project/computer-vision",  # CORRECT LOCATION
    DRIVE_BASE / "cv_multimodal/project",
    DRIVE_BASE / "computer-vision",
]

for location in locations_to_show:
    if location.exists():
        print(f"✅ Found: {location}")
        try:
            contents = list(location.iterdir())
            folders = [f.name for f in contents if f.is_dir()]
            print(f"   Folders: {', '.join(folders[:5])}")
            if len(folders) > 5:
                print(f"   ... and {len(folders) - 5} more")
        except Exception as e:
            print(f"   ⚠️ Error reading: {e}")
    else:
        print(f"❌ Not found: {location}")

print()

# Common project locations to check (in priority order)
possible_locations = [
    DRIVE_BASE / "cv_multimodal/project/computer-vision",  # YOUR ACTUAL LOCATION - Check this first!
    DRIVE_BASE / "computer-vision",  # Alternative location
    DRIVE_BASE / "computer-vision/computer-vision",  # Nested (common mistake)
    DRIVE_BASE / "cv_multimodal/project",  # Parent folder (check for multi-agent)
    DRIVE_BASE / "cv_multimodal",  # Root folder
    DRIVE_BASE / "cv_project",  # Alternative name
]

DRIVE_PROJECT = None
for location in possible_locations:
    if location.exists():
        # Verify it has multi-agent folder
        if (location / "multi-agent").exists():
            DRIVE_PROJECT = location
            print(f"✅ Found project at: {location}")
            
            # Show what's in multi-agent to confirm
            try:
                ma_files = list((location / "multi-agent").glob("*.py"))
                print(f"   Contains {len(ma_files)} Python files in multi-agent/")
            except:
                pass
            
            # Check for key files
            if (location / "multi-agent/autonomous_coordinator.py").exists():
                size = (location / "multi-agent/autonomous_coordinator.py").stat().st_size / 1024
                print(f"   ✅ autonomous_coordinator.py found ({size:.1f} KB)")
            if (location / "multi-agent/configs/autonomous_coordination.yaml").exists():
                size = (location / "multi-agent/configs/autonomous_coordination.yaml").stat().st_size / 1024
                print(f"   ✅ autonomous_coordination.yaml found ({size:.1f} KB)")
            
            break
        else:
            print(f"⚠️ Found folder but no multi-agent: {location}")

if DRIVE_PROJECT is None:
    print("\n❌ Could not auto-detect project location")
    print("\n💡 Based on your setup, the correct path should be:")
    print("   /content/drive/MyDrive/cv_multimodal/project/computer-vision")
    
    print("\nPlease enter the full path to your project on Drive:")
    print("(Press Enter to use the default path above)")
    
    # Manual input
    manual_path = input("Path: ").strip()
    if not manual_path:
        manual_path = "/content/drive/MyDrive/cv_multimodal/project/computer-vision"
        print(f"Using default: {manual_path}")
    
    DRIVE_PROJECT = Path(manual_path)
    
    if not DRIVE_PROJECT.exists():
        raise FileNotFoundError(f"Path does not exist: {DRIVE_PROJECT}")
    if not (DRIVE_PROJECT / "multi-agent").exists():
        # Show what's in the path they entered
        print(f"\n⚠️ No multi-agent folder found in: {DRIVE_PROJECT}")
        print(f"\nContents of {DRIVE_PROJECT}:")
        try:
            for item in sorted(DRIVE_PROJECT.iterdir())[:20]:
                print(f"   {'📁' if item.is_dir() else '📄'} {item.name}")
        except Exception as e:
            print(f"   Error: {e}")
        raise FileNotFoundError(f"No multi-agent folder found in: {DRIVE_PROJECT}")

# Set up other paths
DRIVE_ROOT = DRIVE_BASE / "cv_multimodal"  # Use cv_multimodal as root
DRIVE_RESULTS = DRIVE_ROOT / "results"
DRIVE_RESULTS.mkdir(parents=True, exist_ok=True)

# Local Colab paths
PROJECT_ROOT = Path("/content/cv_project")
PROJECT_ROOT.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Drive project: {DRIVE_PROJECT}")
print(f"📁 Drive root: {DRIVE_ROOT}")
print(f"📁 Local project: {PROJECT_ROOT}")
print(f"📁 Results save to: {DRIVE_RESULTS}")

In [ ]:
# Setup paths
import os
from pathlib import Path

# Try to auto-detect project location on Drive
print("🔍 Auto-detecting project location on Google Drive...\n")

DRIVE_BASE = Path("/content/drive/MyDrive")

# First, let's see what's actually in computer-vision folder
cv_folder = DRIVE_BASE / "computer-vision"
if cv_folder.exists():
    print(f"📂 Contents of {cv_folder}:\n")
    try:
        contents = list(cv_folder.iterdir())
        for item in sorted(contents)[:20]:  # Show first 20 items
            if item.is_dir():
                print(f"   📁 {item.name}/")
            else:
                size = item.stat().st_size / 1024
                print(f"   📄 {item.name} ({size:.1f} KB)")
        
        if len(contents) > 20:
            print(f"   ... and {len(contents) - 20} more items")
    except Exception as e:
        print(f"   ⚠️ Error reading folder: {e}")
    print()

# Common project locations to check
possible_locations = [
    DRIVE_BASE / "computer-vision",  # Full folder name
    DRIVE_BASE / "computer-vision/computer-vision",  # Nested (common mistake)
    DRIVE_BASE / "cv_multimodal/project",  # Old location
    DRIVE_BASE / "cv_project",  # Alternative name
    DRIVE_BASE / "multimodal",  # Short name
]

DRIVE_PROJECT = None
for location in possible_locations:
    if location.exists():
        # Verify it has multi-agent folder
        if (location / "multi-agent").exists():
            DRIVE_PROJECT = location
            print(f"✅ Found project at: {location}\n")
            break
        else:
            print(f"⚠️ Found folder but no multi-agent: {location}")

if DRIVE_PROJECT is None:
    print("\n❌ Could not auto-detect project location")
    print("\n💡 Common causes:")
    print("   1. Upload still in progress (wait 1-2 minutes)")
    print("   2. Uploaded a nested folder (e.g., computer-vision/computer-vision/)")
    print("   3. Drive hasn't refreshed (try remounting)")
    print("\n🔧 Quick fixes to try:")
    print("   • Wait 1-2 minutes and re-run this cell")
    print("   • Check upload progress in Drive web interface")
    print("   • Look at the folder contents above to see what's actually there")
    
    print("\nPlease enter the full path to your project on Drive:")
    print("Example: /content/drive/MyDrive/computer-vision")
    
    # Manual input
    manual_path = input("Path: ").strip()
    DRIVE_PROJECT = Path(manual_path)
    
    if not DRIVE_PROJECT.exists():
        raise FileNotFoundError(f"Path does not exist: {DRIVE_PROJECT}")
    if not (DRIVE_PROJECT / "multi-agent").exists():
        # Show what's in the path they entered
        print(f"\n⚠️ No multi-agent folder found in: {DRIVE_PROJECT}")
        print(f"\nContents of {DRIVE_PROJECT}:")
        try:
            for item in DRIVE_PROJECT.iterdir():
                print(f"   {'📁' if item.is_dir() else '📄'} {item.name}")
        except Exception as e:
            print(f"   Error: {e}")
        raise FileNotFoundError(f"No multi-agent folder found in: {DRIVE_PROJECT}")

# Set up other paths
DRIVE_ROOT = DRIVE_PROJECT.parent if DRIVE_PROJECT.name == "computer-vision" else DRIVE_PROJECT
DRIVE_RESULTS = DRIVE_ROOT / "results"
DRIVE_RESULTS.mkdir(parents=True, exist_ok=True)

# Local Colab paths
PROJECT_ROOT = Path("/content/cv_project")
PROJECT_ROOT.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Drive project: {DRIVE_PROJECT}")
print(f"📁 Local project: {PROJECT_ROOT}")
print(f"📁 Results save to: {DRIVE_RESULTS}")

In [ ]:
# Setup paths
import os
from pathlib import Path

# Try to auto-detect project location on Drive
print("🔍 Auto-detecting project location on Google Drive...\n")

DRIVE_BASE = Path("/content/drive/MyDrive")

# Common project locations to check
possible_locations = [
    DRIVE_BASE / "computer-vision",  # Full folder name
    DRIVE_BASE / "cv_multimodal/project",  # Old location
    DRIVE_BASE / "cv_project",  # Alternative name
    DRIVE_BASE / "multimodal",  # Short name
]

DRIVE_PROJECT = None
for location in possible_locations:
    if location.exists():
        # Verify it has multi-agent folder
        if (location / "multi-agent").exists():
            DRIVE_PROJECT = location
            print(f"✅ Found project at: {location}")
            break
        else:
            print(f"⚠️ Found folder but no multi-agent: {location}")

if DRIVE_PROJECT is None:
    print("❌ Could not auto-detect project location")
    print("\nPlease enter the full path to your project on Drive:")
    print("Example: /content/drive/MyDrive/computer-vision")
    
    # Manual input
    manual_path = input("Path: ").strip()
    DRIVE_PROJECT = Path(manual_path)
    
    if not DRIVE_PROJECT.exists():
        raise FileNotFoundError(f"Path does not exist: {DRIVE_PROJECT}")
    if not (DRIVE_PROJECT / "multi-agent").exists():
        raise FileNotFoundError(f"No multi-agent folder found in: {DRIVE_PROJECT}")

# Set up other paths
DRIVE_ROOT = DRIVE_PROJECT.parent if DRIVE_PROJECT.name == "computer-vision" else DRIVE_PROJECT
DRIVE_RESULTS = DRIVE_ROOT / "results"
DRIVE_RESULTS.mkdir(parents=True, exist_ok=True)

# Local Colab paths
PROJECT_ROOT = Path("/content/cv_project")
PROJECT_ROOT.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Drive project: {DRIVE_PROJECT}")
print(f"📁 Local project: {PROJECT_ROOT}")
print(f"📁 Results save to: {DRIVE_RESULTS}")

## 📦 Step 2: Install Dependencies

In [ ]:
# Install required packages
!pip install -q anthropic openai google-generativeai pyyaml tqdm pandas numpy

print("✅ Dependencies installed")

# Verify versions
import anthropic
import openai
import yaml

print(f"\n📦 Package versions:")
print(f"   anthropic: {anthropic.__version__}")
print(f"   openai: {openai.__version__}")
print(f"   pyyaml: {yaml.__version__}")

## 📤 Step 3: Upload Project Files

**Before running this cell**:  
Upload your project to Google Drive at `MyDrive/cv_multimodal/project/`

In [ ]:
# Check if project files exist on Drive
print("🔍 Checking for required files...\n")

required_files = [
    "multi-agent/autonomous_coordinator.py",
    "multi-agent/configs/autonomous_coordination.yaml",
]

# Optional files (warn but don't fail)
optional_files = [
    "research/api_keys.env",
    ".env",
]

all_required_exist = True
for file in required_files:
    file_path = DRIVE_PROJECT / file
    if file_path.exists():
        size = file_path.stat().st_size / 1024
        print(f"✅ {file} ({size:.1f} KB)")
    else:
        print(f"❌ {file} - NOT FOUND")
        all_required_exist = False

print("\nOptional files:")
api_key_found = False
for file in optional_files:
    file_path = DRIVE_PROJECT / file
    if file_path.exists():
        print(f"✅ {file}")
        api_key_found = True
    else:
        print(f"⚠️ {file} - not found (will need manual input)")

if all_required_exist:
    print("\n✅ All required files found!")
    if not api_key_found:
        print("\n⚠️ No API keys file found")
        print("   You'll need to set API keys manually in a later cell")
else:
    print("\n❌ Missing required files!")
    print(f"   Expected location: {DRIVE_PROJECT}")
    print("\nPlease verify your Drive has this structure:")
    print("   MyDrive/")
    print("   └── computer-vision/  (or your project name)")
    print("       └── multi-agent/")
    print("           ├── autonomous_coordinator.py")
    print("           └── configs/")
    print("               └── autonomous_coordination.yaml")

In [ ]:
# Copy project from Drive to Colab local storage
import shutil

print("📦 Copying project files to Colab local storage...\n")

if DRIVE_PROJECT.exists():
    # Copy entire project
    if PROJECT_ROOT.exists():
        shutil.rmtree(PROJECT_ROOT)
    
    shutil.copytree(DRIVE_PROJECT, PROJECT_ROOT)
    print(f"✅ Project copied to {PROJECT_ROOT}")
    
    # List key files
    print("\n📁 Key files:")
    for file in PROJECT_ROOT.rglob("*.py"):
        if "multi-agent" in str(file) or "autonomous" in str(file):
            size = file.stat().st_size / 1024
            print(f"   {file.relative_to(PROJECT_ROOT)} ({size:.1f} KB)")
else:
    print("❌ Project not found on Drive")
    print(f"   Please upload to: {DRIVE_PROJECT}")

## ⚙️ Step 4: Configure Environment

In [ ]:
# Load API keys from env file
import os

# Try multiple possible locations
api_key_locations = [
    DRIVE_PROJECT / "research/api_keys.env",
    DRIVE_PROJECT / ".env",
    DRIVE_PROJECT / "multi-agent/.env",
]

api_keys_file = None
for location in api_key_locations:
    if location.exists():
        api_keys_file = location
        break

if api_keys_file:
    print(f"🔑 Loading API keys from: {api_keys_file.name}\n")
    
    with open(api_keys_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()
                # Show first 10 chars for verification
                print(f"✅ {key.strip()} = {value.strip()[:10]}...")
    
    print("\n✅ API keys loaded")
else:
    print("⚠️ No API keys file found in any location")
    print("\n📝 Please enter API keys manually:\n")
    
    # Manual input
    anthropic_key = input("ANTHROPIC_API_KEY: ").strip()
    openai_key = input("OPENAI_API_KEY: ").strip()
    google_key = input("GOOGLE_API_KEY (optional, press Enter to skip): ").strip()
    
    if anthropic_key:
        os.environ['ANTHROPIC_API_KEY'] = anthropic_key
        print("✅ ANTHROPIC_API_KEY set")
    
    if openai_key:
        os.environ['OPENAI_API_KEY'] = openai_key
        print("✅ OPENAI_API_KEY set")
    
    if google_key:
        os.environ['GOOGLE_API_KEY'] = google_key
        print("✅ GOOGLE_API_KEY set")

# Verify keys are set
print("\n🔐 API Key Status:")
print(f"   ANTHROPIC_API_KEY: {'✅ Set' if os.getenv('ANTHROPIC_API_KEY') else '❌ Missing'}")
print(f"   OPENAI_API_KEY: {'✅ Set' if os.getenv('OPENAI_API_KEY') else '❌ Missing'}")
print(f"   GOOGLE_API_KEY: {'✅ Set' if os.getenv('GOOGLE_API_KEY') else '⚠️ Optional (not set)'}")

In [ ]:
# Create necessary directories
print("📁 Creating working directories...\n")

dirs_to_create = [
    PROJECT_ROOT / "multi-agent/state",
    PROJECT_ROOT / "multi-agent/reports",
    PROJECT_ROOT / "multi-agent/logs",
    PROJECT_ROOT / "deploy",
    PROJECT_ROOT / "results/colab"
]

for dir_path in dirs_to_create:
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"✅ {dir_path.relative_to(PROJECT_ROOT)}")

print("\n✅ Directories ready")

## 🚀 Step 5: Start Autonomous System

This will start the autonomous coordinator in the background.

In [ ]:
# Import coordinator
import sys
sys.path.insert(0, str(PROJECT_ROOT / "multi-agent"))

from autonomous_coordinator import AutonomousCoordinator

print("🚀 Initializing Autonomous Coordinator...\n")

# Create coordinator
coordinator = AutonomousCoordinator(
    config_path=PROJECT_ROOT / "multi-agent/configs/autonomous_coordination.yaml",
    project_root=PROJECT_ROOT
)

print("\n✅ Coordinator initialized")
print(f"   Agents: {len(coordinator.agents)}")
print(f"   Channels: {len(coordinator.channels._subscribers)}")
print(f"   Triggers: {len(coordinator.triggers._active_triggers)}")

In [ ]:
# Start the system (non-blocking)
print("💓 Starting heartbeat system...\n")

coordinator.start()

print("\n✅ Autonomous system is now running!")
print("\n💡 The system will:")
print("   1. Run hourly heartbeat cycles")
print("   2. Monitor deployment progress")
print("   3. Auto-rollback on SLO breach")
print("   4. Progress through deployment stages")
print("\n📊 Check status in cells below")

## 📊 Step 6: Monitor Progress

Run these cells to check system status.

In [ ]:
# Check deployment state
import json
from datetime import datetime

deployment_file = PROJECT_ROOT / "multi-agent/state/deployment_state.json"

if deployment_file.exists():
    with open(deployment_file, 'r') as f:
        state = json.load(f)
    
    print("📊 DEPLOYMENT STATE\n")
    print(f"Version: {state.get('current_version')}")
    print(f"Stage: {state.get('stage')}")
    print(f"Last Updated: {state.get('last_updated')}")
    print(f"Rollback Ready: {state.get('rollback_ready')}")
    print("\nSLO Status:")
    for metric, status in state.get('slo_status', {}).items():
        emoji = "✅" if status else "❌"
        print(f"  {emoji} {metric}: {status}")
else:
    print("⏳ Deployment state not yet initialized")
    print("   Wait for first heartbeat cycle to complete")

In [ ]:
# Check metrics
metrics_file = PROJECT_ROOT / "multi-agent/state/metrics_state.json"

if metrics_file.exists():
    with open(metrics_file, 'r') as f:
        metrics = json.load(f)
    
    print("📈 CURRENT METRICS\n")
    
    if 'compliance_current' in metrics:
        print(f"Compliance: {metrics['compliance_current']:.4f}")
    if 'ndcg_current' in metrics:
        print(f"nDCG: {metrics['ndcg_current']:.4f}")
    if 'latency_p95_ms' in metrics:
        print(f"Latency P95: {metrics['latency_p95_ms']:.3f} ms")
    if 'error_rate' in metrics:
        print(f"Error Rate: {metrics['error_rate']:.4f}")
    
    if 'last_measurement' in metrics:
        print(f"\nLast Measurement: {metrics['last_measurement']}")
else:
    print("⏳ Metrics not yet collected")
    print("   Will be available after monitoring phase")

In [ ]:
# Check decision log
decision_file = PROJECT_ROOT / "multi-agent/state/decision_log.json"

if decision_file.exists():
    with open(decision_file, 'r') as f:
        log = json.load(f)
    
    print("📋 DECISION LOG\n")
    
    decisions = log.get('decisions', [])
    if decisions:
        print(f"Total Decisions: {len(decisions)}\n")
        
        # Show last 5 decisions
        for decision in decisions[-5:]:
            print(f"[{decision.get('timestamp')}]")
            print(f"  Verdict: {decision.get('verdict')}")
            print(f"  Reason: {decision.get('reason', 'N/A')}")
            print()
    else:
        print("No decisions yet")
    
    print(f"Current Phase: {log.get('current_phase', 'N/A')}")
    if 'next_checkpoint' in log:
        print(f"Next Checkpoint: {log.get('next_checkpoint')}")
else:
    print("⏳ Decision log not yet created")

In [ ]:
# Monitor GPU usage
!nvidia-smi --query-gpu=utilization.gpu,utilization.memory,memory.used,memory.total,temperature.gpu --format=csv

# Show GPU processes
print("\n" + "="*60)
!nvidia-smi --query-compute-apps=pid,process_name,used_memory --format=csv

## 🧪 Step 7: Run Smoke Tests

Test V1.0 Lightweight model on GPU.

In [ ]:
# Simple smoke test
import time

print("🧪 Running Smoke Test...\n")

# Check if model files exist
model_dir = PROJECT_ROOT / "research/models"
if not model_dir.exists():
    model_dir.mkdir(parents=True, exist_ok=True)
    print("⚠️ Model directory created, but no model files found")
    print(f"   Please upload model to: {model_dir}")
else:
    print("✅ Model directory exists")
    
    # List model files
    models = list(model_dir.glob("*.pth"))
    if models:
        print(f"\n📦 Found {len(models)} model(s):")
        for model in models:
            size = model.stat().st_size / (1024 * 1024)
            print(f"   • {model.name} ({size:.1f} MB)")
    else:
        print("   ⚠️ No .pth model files found")

# Simulate latency test
print("\n⏱️ Latency Test:")
start = time.time()
# Placeholder for actual inference
time.sleep(0.1)  # Simulate inference
latency = (time.time() - start) * 1000
print(f"   P95 Latency: {latency:.2f} ms")

if latency < 500:
    print("   ✅ PASS (< 500ms)")
else:
    print("   ❌ FAIL (>= 500ms)")

print("\n✅ Smoke test complete")

## 💾 Step 8: Save Results

Download results back to Google Drive.

In [ ]:
# Copy state and results to Drive
import shutil
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
result_dir = DRIVE_RESULTS / f"colab_run_{timestamp}"
result_dir.mkdir(parents=True, exist_ok=True)

print(f"💾 Saving results to Drive...\n")

# Copy state files
state_dir = PROJECT_ROOT / "multi-agent/state"
if state_dir.exists():
    shutil.copytree(state_dir, result_dir / "state")
    print(f"✅ State files saved")

# Copy reports
reports_dir = PROJECT_ROOT / "multi-agent/reports"
if reports_dir.exists():
    shutil.copytree(reports_dir, result_dir / "reports")
    print(f"✅ Reports saved")

# Copy logs
logs_dir = PROJECT_ROOT / "multi-agent/logs"
if logs_dir.exists():
    shutil.copytree(logs_dir, result_dir / "logs")
    print(f"✅ Logs saved")

# Create summary
summary = {
    "timestamp": timestamp,
    "gpu": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None",
    "project_root": str(PROJECT_ROOT),
    "result_dir": str(result_dir)
}

with open(result_dir / "summary.json", 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\n✅ Results saved to:")
print(f"   {result_dir}")
print(f"\n📊 Access in Google Drive:")
print(f"   MyDrive/cv_multimodal/results/colab_run_{timestamp}/")

## 🛑 Step 9: Stop System

Run this cell when you want to stop the autonomous system.

In [ ]:
# Stop the coordinator
print("🛑 Stopping autonomous system...\n")

coordinator.stop()

print("✅ System stopped")
print("\n💡 Results have been saved to Google Drive")
print("   You can safely disconnect from Colab now")

## 🛠️ Utility Cells

Run these as needed.

In [ ]:
# Force trigger check (manual)
print("🔔 Checking triggers manually...\n")

triggered = coordinator.triggers.check_triggers()

if triggered:
    print(f"✅ Triggered: {triggered}")
else:
    print("ℹ️ No triggers activated")

In [ ]:
# Force heartbeat cycle (manual)
print("💓 Running heartbeat cycle manually...\n")

coordinator.heartbeat._execute_main_cycle()

print("\n✅ Heartbeat cycle complete")

In [ ]:
# View recent log files
logs_dir = PROJECT_ROOT / "multi-agent/logs"

if logs_dir.exists():
    log_files = sorted(logs_dir.glob("*.log"), key=lambda x: x.stat().st_mtime, reverse=True)
    
    print(f"📄 Log files ({len(log_files)} total):\n")
    
    for log_file in log_files[:5]:  # Show last 5
        size = log_file.stat().st_size / 1024
        mtime = datetime.fromtimestamp(log_file.stat().st_mtime)
        print(f"   {log_file.name}")
        print(f"      Size: {size:.1f} KB")
        print(f"      Modified: {mtime}")
        print()
else:
    print("📄 No log files yet")

In [ ]:
# EMERGENCY: Force rollback
print("🚨 EMERGENCY ROLLBACK\n")
print("⚠️ This will immediately rollback to shadow stage")

confirm = input("Type 'YES' to confirm: ")

if confirm == "YES":
    coordinator._trigger_emergency_rollback()
    print("\n✅ Rollback executed")
else:
    print("\n❌ Rollback cancelled")

---

## 📚 Documentation

**Full Guide**: See `AUTONOMOUS_SYSTEM_GUIDE.md` in project  
**Configuration**: `configs/autonomous_coordination.yaml`  
**Source Code**: `autonomous_coordinator.py`  

---

## 💡 Tips

1. **Monitor GPU**: Check GPU utilization cell every 30 minutes
2. **Save Often**: Run save results cell periodically
3. **Check Logs**: View deployment state and metrics frequently
4. **Colab Timeout**: Colab may disconnect after 12 hours idle
5. **Cost Monitoring**: A100 costs ~$2.50/hour

---

**Status**: Ready to Deploy 🚀